# 提升与Boost

提升是一个机器学习技术，可以用于回归和分类问题，它每一步产生一个弱预测模型，如决策树，并加权累加到总模型中；如果每一步的弱预测模型生成都是依据损失函数的梯度方向，则称之为梯度提升Gradient boosting

梯度提升算法首先给定一个目标损失函数，它的定义域是所有可行的若函数集合(基函数)；提升算法通过迭代的选择一个负梯度方向上的基函数来逐渐逼近局部最小值

提升的理论意义：如果一个问题存在弱分类器，则可以通过提升的办法得到强分类器

工业界正确率达到90%才是刊用的。

提升算法主要包括GDBT、XGBoost以及AdaBoost三个

#1 思考
随机森林的决策树分别采样建立，相对独立，假定当前得到了m-1棵决策树，是否可以通过现有样本和决策树信息，对第m棵决策树的建立产生有益的影响？

#2 算法
- 给定输入向量X和输出变量Y组成的若干训练样本$(x_1,y_1),(x_2,y_1),...,(x_n,y_n)$，目标就是找到近似函数$\hat{F}(\overrightarrow{X})$，使得损失函数$L(Y,F(X))$的损失值最小

- $L(Y,F(X))$的典型定义以下两种
$$	
\begin{cases}
L(Y,F(\overrightarrow{x}))=\frac{1}{2}(y-F(\overrightarrow{x}))^2\\	
L(Y,F(\overrightarrow{x}))=|y-F(\overrightarrow{x})|
\end{cases}
$$
- 假定最优函数为$F^*(\overrightarrow{x})$，即$F^*(\overrightarrow{x})=argmin_FE_{x,y}[L(y,F(\overrightarrow{x}))]$
- 假定F(X)是一族基函数$f_i(X)$的加权和$F(\overrightarrow{x})=\sum_{i=1}^M\gamma_if_i(x)+const$

对于式子1来说，$F(\overrightarrow{x})$就是均值，对于式子2来说，$F(\overrightarrow{x})$就是中位数

#3 一阶信息推导-GBDT
梯度提升方法寻找最优解F(x)，使得损失函数在训练集上的期望最小.

有两种方法，一种是残差，一种就是梯度提升

##3.1 常函数
$F_0(\overrightarrow{x})=argmin_{\gamma}\sum_{i=1}^nL(y_i,\gamma)$

对于m=1到M
- 计算伪残差$\gamma_{im}=[\frac{\partial{L(y_i,F(\overrightarrow{x_i}))}}{\partial{F(\overrightarrow{x_i})}}]_{F(\overrightarrow{x})=F_{m-1}(\overrightarrow{x})}$, i=1,2,...,n
- 使用数据$(\overrightarrow{x_i}, \gamma_{im})_{i=1}^n$计算拟合残差的基函数$f_m(x)$
- 计算步长$\gamma_m=argmin_{\gamma}\sum_{i=1}^nL(y_i,F_{m-1}(\overrightarrow{x_i})-\gamma \bullet f_m(\overrightarrow{x_i}))$，这是一个一维优化问题
- 更新模型$F_m(\overrightarrow{x})=F_{m-1}(\overrightarrow{x})-\gamma_mf_m(\overrightarrow{x_i})$

##3.2 贪心推导
以贪心的思路扩展得到$F_m(X)$:$F_m(\overrightarrow(x))=F_{m-1}(\overrightarrow{x})+argmin_{f \in H}\sum_{i=1}^nL(y_i, F_{m-1}(\overrightarrow{x_i})+f(\overrightarrow{x_i}))$

梯度提升的典型基函数就是决策树，尤其是CART决策树。在第m步的梯度提升是根据伪残差数据计算决策树$t_m(X)$。令树$t_m(X)$的叶节点数目为J，即树$t_m(X)$将输入空间划分为J个不相交区域$R_{1m},R_{2m},...,R_{Jm}$，并且决策树$t_m(X)$可以在每个区域中给出某个类型的确定性预测。使用记号$I(X)$，对于输入x，$t_m(X)$为$t_m(\overrightarrow{x})=\sum_{j=1}^Jb_{jm}I(\overrightarrow{x} \in R_{jm})$，其中，$b_{jm}$是样本X在区域$R_{jm}$的预测值

#4 二阶信息推导-XGBoost
使用一阶导总是会涉及到学习率$\gamma$，我们考虑二阶导

假设有样本X:$\{\overrightarrow{x_1},\overrightarrow{x_2},...,\overrightarrow{x_m}\}$，以及对应的Y个真实值$\{y_1,y_2,....,y_m\}$。目前我们已经找到了t-1个决策树$\{T_1,T_2,...,T_{t-1}\}$，以及对应的t-1个学习率$\{\\alpha_1,\alpha_2,...,\alpha_{t-1}\}$。那么对于任意一个样本$\overrightarrow{x_i}$，我们总能算出一个预测值$\hat{y_i}=\alpha_1T_1(x_i)+\alpha_2T_2(x_i)+...+\alpha_{t-1}T_{t-1}(x_i)$。我们使用符号$\hat{y}_{t-1}^{(i)}$来表示使用t-1棵决策树计算出来的第i个样本的预测值，那么我们就有了一组数据$\{(x^{(1)}, \hat{y}_{t-1}^{(1)}), (x^{(2)}, \hat{y}_{t-1}^{(2)}),...,(x^{(m)}, \hat{y}_{t-1}^{(m)})\}$。现在我们要考虑的是怎么计算$T_t(X)$以及$\alpha_t$

##4.1 目标函数
$J(f_t)=\sum_{i=1}^nL(y_i,\hat{y}_{t-1}^{(i)} + f_t(x_i))+\Omega(f_t)+C$，本身f是未知的

##4.2 Taylor展式
$f(x+\Delta{x}) \approx f(x)+f'(x)\Delta{x}+\frac{1}{2}f''(x)\Delta{x}^2$，我们可以看出来$\hat{y}_{t-1}^{(i)}$相当于Taylor展式中的x，$f_t(x_i)$相当于$\Delta{x}$。

令$g_i=\frac{\partial{L(y_i, \hat{y}_{t-1}^{(i)})}}{\partial{\hat{y}_{t-1}^{(i)}}}$,$h_i=\frac{\partial^2{L(y_i, \hat{y}_{t-1}^{(i)})}}{\partial{\hat{y}_{t-1}^{(i)}}}$

由于$\hat{y}_{t-1}^{(i)}$是可以计算出来的，损失函数L是已知的，所以$g_i,h_i$是可以提前计算出来的，所有就有$J(f_t) \approx \sum_{i=1}^n[L(y_i, \hat{y}_{t-1}^{(i)})+g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)]+\Omega{(f_t)}+C$

##4.3 举例说明
![images](images/05.png)
这是一个是否喜欢计算机游戏的例子:$f_t(x)=\omega_{q(x)}, \omega \in R^T, q:R^d \{1,2,3,...,T\}$，比如q("boy")=1，因为boy在叶节点1上，也可以是q("boy")=+2，因为叶节点1的权重是+2

##4.4 正则项的定义
决策树的复杂度可考虑叶节点数和叶权值，如使用叶节点总数和叶权值平方和的加权。意思就是说，如果我们觉得一个树很复杂，那么就是说树的叶子节点比较多，并且每个叶子节点的权值比较大。所以我们可以将二阶导数的$\Omega(f_t)$写成这样的形式$\Omega(f_t)=\gamma \bullet T_t + \lambda \bullet \frac{1}{2}\sum_{j=1}^T\omega_j^2$，其中$\gamma,\lambda$是超参数，$T_t$是叶子节点的个数，$\omega$是叶子结点的权重。比如上例中，有3个节点，每个节点的权重为2，0.1，-1,所以$\Omega(f_t)=3\gamma+\frac{1}{2}\lambda(4+0.01+1)$。这就是正则项。这么表示正则项不是唯一的，只不过这么表示比较直观，好推导

对于二阶导，$J(f_t) \approx \sum_{i=1}^n[L(y_i, \hat{y}_{t-1}^{(i)})+g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)]+\Omega{(f_t)}+C$

$\because L(y_i, \hat{y}_{t-1}^{(i)})$是一个常数，所以可以和最后的C进行合并

$\Rightarrow J(f_t)\approx \sum_{i=1}^n[g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)]+\Omega{(f_t)}+C$

$J(f_t)\approx \sum_{i=1}^n[g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)]+\gamma \bullet T + \lambda \bullet \frac{1}{2}\sum_{j=1}^T\omega_j^2+C$

$\because f_t(x_i)=\omega_{q(x_i)}, \therefore J(f_t)\approx \sum_{i=1}^n[g_i\omega_{q(x_i)}+\frac{1}{2}h_i\omega_{q(x_i)}^2]+\gamma \bullet T + \lambda \bullet \frac{1}{2}\sum_{j=1}^T\omega_j^2+C$

到了关键点$\sum_{i=1}^n[g_i\omega_{q(x_i)}+\frac{1}{2}h_i\omega_{q(x_i)}^2]$这个是对于样本从1到n进行操作，那么最终这些样本都会落在不同的叶子节点上，而$\omega$是叶子的权值。样本相加，其实就相当于所有叶子结点权值和预测值乘积的相加

$\therefore \Rightarrow \sum_{j=1}^T[(\sum_{i \in I_j}g_i)\omega_j+\frac{1}{2}(\sum_{i \in I_j}h_i)\omega_j^2] +\gamma \bullet T + \lambda \bullet \frac{1}{2}\sum_{j=1}^T\omega_j^2+C$

$\Rightarrow \sum_{j=1}^T[(\sum_{i \in I_j}g_i)\omega_j+\frac{1}{2}(\sum_{i \in I_j}h_i + \lambda)\omega_j^2] +\gamma \bullet T +C$

令$G_j=\sum_{i \in I_j}g_i, H_j=\sum_{i \in I_j}h_i$

$\Rightarrow J(f_t)=\sum_{j=1}^T[G_j\omega_j+\frac{1}{2}(H_j+\lambda)\omega_j^2]+\gamma \bullet T + C$

只有$\omega$是未知，求偏导有$\frac{\partial{J(f_t)}}{\partial{\omega_j}}=G_j+(H_j+\lambda)\omega_j = 0$

$\Rightarrow \omega_j=-\frac{G_j}{H_j+\lambda}$

将$\omega_j$回代入目标函数得到$J(f_t)=-\frac{1}{2}\sum_{j=1}^T\frac{G_j^2}{H_j+\lambda}+\gamma \bullet T$

![images](images/06.png)


##4.5 子树划分
由于我们知道了损失值$J(f_t)=-\frac{1}{2}\sum_{j=1}^T\frac{G_j^2}{H_j+\lambda}+\gamma \bullet T$，那么对于某可行划分，计算划分后的J(f)，找到所有的可行划分，选择J(f)降低最小的分割点
![images](images/07.png)

##4.6 总结
1. 传统GBDT以CART作为基分类器，xgboost还支持线性分类器，这个时候xgboost相当于带L1和L2正则化项的逻辑斯蒂回归（分类问题）或者线性回归（回归问题）
2. 传统GBDT在优化时只用到一阶导数信息，xgboost则对代价函数进行了二阶泰勒展开，同时用到了一阶和二阶导数。顺便提一下，xgboost工具支持自定义代价函数，只要函数可一阶和二阶求导
3. xgboost在代价函数里加入了正则项，用于控制模型的复杂度。正则项里包含了树的叶子节点个数、每个叶子节点上输出的score的L2模的平方和。从Bias-variance tradeoff角度来讲，正则项降低了模型variance，使学习出来的模型更加简单，防止过拟合，这也是xgboost优于传统GBDT的一个特性 
4. 在每一步tree boosting之后增加了一个参数n（权重），通过这种方式来减小每棵树的影响力，给后面的树提供空间去优化模型
5. 列(特征)抽样，说是从随机森林那边学习来的，防止过拟合的效果比传统的行抽样还好（行抽样功能也有），并且有利于后面提到的并行化处理算法
6. 并行化处理

#5 举例

##5.1 预测年龄-GBDT
训练集：(A, 14岁)、(B，16岁)、(C, 24岁)、(D, 26岁)；
训练数据的均值：20岁；(这个很重要，因为GBDT与i开始需要设置预测的均值，这样后面才会有残差！)
决策树的个数：2棵；
每个样本的特征有两个：购物金额是否小于1K；经常去百度提问还是回答;

###5.1.1 第一棵决策树
首先，输入初值20岁，根据第一个特征(具体选择哪些特征可以根据信息增益来计算选择)，可以把4个样本分成两类，一类是购物金额<=1K，一类是>=1K的。假如这个时候我们就停止了第一棵树的学习，这时我们就可以统计一下每个叶子中包含哪些样本，这些样本的均值是多少，因为这个时候的均值就要作为所有被分到这个叶子的样本的预测值了。比如AB被分到左叶子，CD被分到右叶子，那么预测的结果就是：AB都是15岁，CD都是25岁。和他们的实际值一看，结果发现出现的残差，ABCD的残差分别是-1, 1, -1, 1。这个残差，我们要作为后面第二棵决策树的学习样本![images](images/08.png)

###5.1.2 第二棵决策树
然后学习第二棵决策树，我们把第一棵的残差样本（A, -1岁）、（B，1岁）、（C, -1岁）、（D, 1岁）输入。此时我们选择的特征是经常去百度提问还是回答。这个时候我们又可以得到两部分，一部分是AC组成了左叶子，另一部分是BD组成的右叶子。那么，经过计算可知左叶子均值为-1，右叶子均值为1. 那么第二棵数的预测结果就是AC都是-1，BD都是1.我们再来计算一下此时的残差，发现ABCD的残差都是0！停止学习~![images](images/09.png)

###5.1.3 预测环节
测试样本：请预测一个购物金额为3k，经常去百度问淘宝相关问题的女生的年龄~

我们提取2个特征：购物金额3k，经常去百度上面问问题；

第一棵树 —> 购物金额大于1k —> 右叶子，初步说明这个女生25岁
第二棵树 —> 经常去百度提问 —> 左叶子，说明这个女生的残差为-1；

叠加前面每棵树得到的结果：25-1=24岁，最终预测结果为24岁~